<div>
<img src="svtLogo.png"/>
</div>
<h1><center>Mathematical Optimization for Engineers</center></h1>
<h2><center>Lab 14 - Uncertainty</center></h2>

We want to optimize the total annualized cost of a heating and electric power system. Three different technologies are present: 
- a gas boiler
- a combined heat and power plant
- a photovoltaic module

We first the the nominal case without uncertanties. 
Next, we will consider a two-stage approach to consider uncertainties in the electricity demand and the power producable via PV. 
Uncertain variables are the solar power and the power demand. 

In [1]:
# import cell
from scipy.optimize import minimize, NonlinearConstraint, Bounds

In [2]:
class Boiler():
    """Boiler 
    Gas in, heat out
    """
    
    def __init__(self):
        self.M = 0.75  
        
    def invest_cost(self, Qdot_nom):
        inv = 100 * Qdot_nom ** self.M
        return inv
    
    def oper_cost(self, Qdot_nom, op_load): 
        cost_gas = 60
        cost_gas_oper = Qdot_nom * cost_gas * op_load
        
        return cost_gas_oper
    
    def heat(self, Qdot_nom, op_load):
        eta_th = 0.9 - (1 - op_load) * 0.05
        return Qdot_nom * op_load * eta_th
    

In [3]:
class CHP():
    """Combined-heat-and-power (CHP) engine 
    Gas in, heat and power out
    """

    def __init__(self):
        self.c_ref = 150
        self.M = 0.85  # [-], cost exponent
        self.cost_gas = 60
    
    def invest_cost(self, Qdot_nom):
        inv = self.c_ref * (Qdot_nom) ** self.M
        return inv
    
    def oper_cost(self, Qdot_nom, op_load): 
        cost_gas_oper = Qdot_nom * op_load * self.cost_gas
        return cost_gas_oper
    
    def elec_out(self, Qdot_nom, op_load):
        eta_el = 0.3 - (1 - op_load) * 0.1
        out_pow = eta_el * Qdot_nom * op_load
        return out_pow
    
    def heat(self, Qdot_nom, op_load): 
        eta_th = 0.6 - (1-op_load) * 0.05  
        return Qdot_nom * eta_th * op_load


In [4]:
class PV:
    """Photovoltaic modules (PV) 
    solar 
    """ 
    
    def __init__(self): 
        self.M = 0.9  # [-], cost exponent
       
    def invest_cost(self, p_nom):
        inv = 200 * p_nom ** self.M
        return inv
    
    def oper_cost(self, out_nom): 
        return 0
    
    def elec_out(self, p_nom, op_load, solar):
        return p_nom * op_load * solar
    

In [5]:
def objective_function(x, PV, Boiler, CHP, scenarios):
    total_cost = 0
    design_PV = x[0]  
    design_boiler = x[1]  
    design_CHP = x[2] 
    
    cost_PV = PV.invest_cost(design_PV) 
    cost_Boiler = Boiler.invest_cost(design_boiler)  
    cost_CHP = CHP.invest_cost(design_CHP) 
        
    total_cost = cost_PV + cost_Boiler + cost_CHP
    operating_cost = 0
    
    iIndexShift = 3
    for idx, iSec in enumerate(scenarios): 
        indexOffset = 3 + idx * iIndexShift
        
        op_cost = Boiler.oper_cost(design_boiler, x[indexOffset]) \
             + CHP.oper_cost(design_CHP, x[indexOffset + 1])
        total_cost = total_cost + iSec["p"] * op_cost
        
   
    return total_cost

In [6]:
def constraint_function(x, PV, Boiler, CHP, scenarios): 
    heat_demand = 200
    c = list  
    c = [0]
    design_PV = x[0]  
    design_boiler = x[1]  
    design_CHP = x[2]  

    # loop over all uncertatintes
    iIndexShift = 3
    for idx, iSec in enumerate(scenarios): 
        indexOffset = 3 + idx * iIndexShift
        elec_demand = iSec["elec"]
        
        # heat demand
        c.append(Boiler.heat(design_boiler, x[indexOffset]) \
             + CHP.heat(design_CHP, x[indexOffset + 1]) - heat_demand)     
        # electricty demand 
        c.append(PV.elec_out(design_PV, x[indexOffset], iSec["solar"])
              + CHP.elec_out(design_CHP, x[indexOffset + 1]) - elec_demand)
    # remove initial list element       
    c.pop(0)
    return c

In [7]:
def print_solution(x):
    print('PV design: ', x[0])
    print('Boiler design: ', x[1])
    print('CHP design: ', x[2])
    

In [8]:
# nominal case
scenario1 = {"p": 1.0, "solar":1.0, "elec": 100}
scenarios = [scenario1] # some base scenario


In [9]:
# now consider different scenarios
myPV = PV()
myBoiler = Boiler()
myCHP = CHP()
cons = lambda x: constraint_function(x, myPV, myBoiler, myCHP, scenarios)
obj = lambda x: objective_function(x, myPV, myBoiler, myCHP, scenarios)
nonlinear_constraints = NonlinearConstraint(cons, 0, 0)
# bounds for operation 0 . 1
x_guess = [200,200,200, 1,1,1 ]
lbs = [0, 0, 0, 0, 0, 0]
ubs = [10000, 10000, 10000, 1, 1, 1]
bnds = Bounds(lbs, ubs)



In [10]:
res = minimize(obj, x_guess, method = 'SLSQP', bounds=bnds,
               constraints = nonlinear_constraints,
               options={"maxiter": 15, 'iprint': 2, 'disp': True})


  NIT    FC           OBJFUN            GNORM
    1     8     6.641731E+04     1.697149E+04
    2    16     4.385067E+04     1.461622E+04
    3    24     3.104611E+04     1.011635E+04
    4    32     3.412252E+04     1.194103E+04
    9    40     3.170808E+04     1.353084E+04
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 31708.08358186242
            Iterations: 9
            Function evaluations: 40
            Gradient evaluations: 5


In [11]:
print(res.x)

[100.00000789 222.22225144   0.           1.           1.
   1.        ]


In [12]:
print_solution(res.x)

PV design:  100.00000788759881
Boiler design:  222.22225144020462
CHP design:  0.0


In [13]:
# nominal 
# uncertanties: power demand and solar power (relative 1.0)
scenario1 = {"p": 0.40, "solar":1.0, "elec": 100}
scenario2 = {"p": 0.3, "solar":1.0, "elec": 120}
scenario3 = {"p": 0.3, "solar":0.5, "elec": 80}

scenarios = [scenario1] # some base scenario
scenarios.append(scenario2)
scenarios.append(scenario3)

print(scenarios)

[{'p': 0.4, 'solar': 1.0, 'elec': 100}, {'p': 0.3, 'solar': 1.0, 'elec': 120}, {'p': 0.3, 'solar': 0.5, 'elec': 80}]


In [14]:
myPV = PV()
myBoiler = Boiler()
myCHP = CHP()
cons = lambda x: constraint_function(x, myPV, myBoiler, myCHP, scenarios)
obj = lambda x: objective_function(x, myPV, myBoiler, myCHP, scenarios)
nonlinear_constraints = NonlinearConstraint(cons, 0, 0)
# bounds for operation 0 . 1
x_guess = [200,200,200, 1,1,1, 1,1,1, 1,1,1 ]
lbs = [0, 0, 0,   0, 0, 0,  0, 0, 0,  0, 0, 0]
ubs = [10000, 10000, 10000, 1, 1, 1, 1, 1, 1, 1, 1, 1]
bnds = Bounds(lbs, ubs)

res = minimize(obj, x_guess, method = 'SLSQP', bounds=bnds,
               constraints = nonlinear_constraints,
               options={"maxiter": 15, 'iprint': 2, 'disp': True})

  NIT    FC           OBJFUN            GNORM
    1    14     6.641731E+04     9.897042E+03
    2    28     4.957814E+04     9.011641E+03
    3    42     5.279361E+04     1.171438E+04
    4    56     5.787383E+04     1.388065E+04
    5    70     5.615782E+04     1.304099E+04
    6    84     5.597843E+04     1.302393E+04
    7    98     5.375643E+04     1.252509E+04
    8   112     5.277753E+04     1.236363E+04
    9   126     5.295557E+04     1.238931E+04
   10   140     5.295484E+04     1.238924E+04
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 52954.835161945666
            Iterations: 10
            Function evaluations: 141
            Gradient evaluations: 10


In [15]:
print_solution(res.x)

PV design:  80.00000000000247
Boiler design:  119.46005043013979
CHP design:  333.3333333333102
